# Extract Sub Grids

Read grid files and generate various subsets.

In [4]:
import sys
from pathlib import Path
import numpy as np
from tqdm import tqdm

sys.path.insert(0, '../python')
from simpython.cmg import gridfile  # type: ignore # pylint: disable=import-error,wrong-import-position
from simpython.common import file_utils  # type: ignore # pylint: disable=import-error,wrong-import-position

In [9]:
import pandas as pd

df = pd.DataFrame([[1,2,3],[4,5,6]])

df

,0,1,2
0,1,2,3
1,4,5,6


In [18]:
Path('./asd#!:\\"@#$%¨&*()////ddd.rst').parent.mkdir(parents=True, exist_ok=True)

## Options

In [19]:
def list_files(folder_path):
    """ List files names in folder """
    try:
        folder_path = Path(folder_path)
        filenames = [file for file in folder_path.iterdir() if file.is_file()]
        filenames.sort()
        return filenames
    except FileNotFoundError:
        print(f"Folder '{folder_path}' not found.")
        return []

In [20]:
geo_folder_path = '../Unisim_iv_2024/grid/2024/prior'
geo_folder_path = Path(geo_folder_path)

### 2D

In [21]:
options = {
    'ni': 47,
    'nj': 39,
    'nk': 291,
    'file_names': ['PERMI', 'PERMJ', 'PERMK', 'POR'],
    'index': [1, 10, 11, 47, 99],
    'subgrids': [],
    'output_folder': '2d/grid'
}

for k in range(41,54):
    options['subgrids'].append( (33,37,12,16,k,k) )
for k in range(75,96):
    options['subgrids'].append( (33,37,12,16,k,k) )
for k in range(206,224):
    options['subgrids'].append( (17,21,19,23,k,k) )
for k in range(229,270):
    options['subgrids'].append( (17,21,19,23,k,k) )

#### Tests

In [22]:
options = {
    'ni': 47,
    'nj': 39,
    'nk': 291,
    'file_names': ['PERMI', 'PERMJ', 'PERMK', 'POR'],
    'index': [27, 14],
    'subgrids': [
        (16,20,11,15,51,51),
        (29,33,17,21,51,51),
        (29,33,17,21,84,84),
        (14,18,22,26,84,84)
    ],
    'output_folder': '2d_test/grid'
}

## Main Functions

In [23]:
def n2ijk(n):
    ni = options['ni']
    nj = options['nj']
    # nk = options['nk']

    k = int(n/(ni*nj)) + 1
    j = int( (n/ni - (k-1)*nj)) + 1
    i = n - (k-1)*ni*nj - (j-1)*ni + 1
    return i,j,k

def ijk2n(i,j,k):
    ni = options['ni']
    nj = options['nj']
    # nk = options['nk']

    return (k-1)*ni*nj + (j-1)*ni + i-1

def get_value(values, i,j=None,k=None):
    if j is None:
        return values[i]
    elif k is not None:
        return values[k-1,j-1,i-1]
    raise ValueError('Invalid option!')

In [24]:
def create_sub_grid(file_path, output_file_path, range_):
    grid = gridfile.GridFile(file_path=file_path)

    ni = options['ni']
    nj = options['nj']
    nk = options['nk']

    if grid.get_number_values() != ni*nj*nk:
        msg = f'Size mismatch: Values read = {grid.get_number_values()}.'
        msg += f' Values expected = {ni*nj*nk}.'
        raise ValueError(msg)

    values = np.array(grid.get_values())
    values_3d = values.reshape(nk, nj,-1)

    def get_subgrid(range_):
        i1, i2, j1, j2, k1, k2 = range_
        i1 -= 1
        j1 -= 1
        k1 -= 1
        return values_3d[k1:k2, j1:j2, i1:i2]

    def write_grid(output_file_path, values, comments):
        gridfile.GridFile.write_(
            output_file_path=output_file_path,
            values=values.reshape(-1),
            keyword=grid.get_keyword(),
            comments=comments)

    i1, i2, j1, j2, k1, k2 = range_
    comments = f'** {file_path.name} subgrid: [{i1}:{i2},{j1}:{j2},{k1}:{k2}]'

    write_grid(
        output_file_path=output_file_path,
        values=get_subgrid(range_),
        comments=comments)

In [25]:
output_folder_path = Path('./'+options['output_folder'])
output_folder_path.mkdir(parents=True, exist_ok=True)

pbar = tqdm(total=len(options['index'])*len(options['file_names'])*len(options['subgrids']))
for i in options['index']:
    for file_name in options['file_names']:
        file_path = geo_folder_path/f'{file_name}_{i}.geo'
        if not file_path.is_file():
            print(f'{file_path} not found!')
            pbar.update(len(options['subgrids']))
            continue
        for k,range_ in enumerate(options['subgrids']):
            new_file_name = f'{file_name}_{i}_{k}.geo'
            create_sub_grid(
                file_path=file_path,
                output_file_path=output_folder_path/new_file_name,
                range_=range_)
            pbar.update(1)
pbar.close()

100%|██████████| 32/32 [00:08<00:00,  3.82it/s]


In [26]:
output_folder_path = Path('./'+options['output_folder'])
output_folder_path.mkdir(parents=True, exist_ok=True)

file_utils.zip_files(
        folder_path=output_folder_path,
        extensions=['.geo'],
        file_name='geo_files',
        delete_original=True)